# 前言

数据源是来自[和鲸社区](https://www.kesci.com/mw/dataset/5ffac64f3441fd001538228b/file)的 1 份虚拟的电商数据集  
需特别说明的是，由于该数据集为模拟生成，部分字段（如订单时间分布、地区消费均衡性等）可能与企业真实业务数据存在差异，仅用于数据分析方法演示与逻辑验证，敬请使用者留意。  
该数据集涵盖 50 万条订单记录，包含订单基础信息（订单 ID、用户 ID、下单时间）、商品属性信息（商品大类、子类、名称、单价）、交易核心指标（购买数量、订单总额）、收货地理信息（省份、城市）及用户画像数据（性别、年龄、是否会员）六大维度。  
基于这些数据，可深入挖掘三大核心方向：  
一是空间与时间维度的消费差异（空间上为各省份销量与销售额分布，时间上为小时级、日级下单规律）；  
二是具体到产品分类的销售业绩分析（计算总GMV，平均客单价，找出TOP 热销商品）；  
三是用户与商品的关联分析（如不同性别 / 年龄用户的消费偏好），同时通过 RFM 模型实现用户分层，对比不同层次用户的消费差异，为精准运营提供数据支撑。

## 1、数据理解与处理

In [2]:
import pandas as pd
data=pd.read_csv("阿迪达斯订单数据.csv")
data.head()

,订单ID,用户ID,下单时间,商品大类,商品子类,商品名称,单价,购买数量,订单总额,收货省份,收货城市,性别,年龄,是否会员
0,318477,9052,2024-01-01 00:00:47.416583,运动鞋,户外鞋,特锐斯山地系列,1241.82,1,1241.82,贵州,安顺,男,35,否
1,112481,4053,2024-01-01 00:02:40.515327,运动服装,上衣,篮球上衣,367.87,1,367.87,北京,北京,女,25,否
2,149467,458,2024-01-01 00:02:45.045488,运动服装,上衣,运动夹克,718.49,1,718.49,上海,上海,女,21,是
3,340464,6811,2024-01-01 00:04:50.465206,运动服装,套装,运动套装,803.27,1,803.27,四川,乐山,女,23,是
4,153344,4947,2024-01-01 00:08:07.517883,运动服装,外套,保暖外套,862.81,1,862.81,浙江,温州,男,23,否


In [3]:
#显示data简要信息，500000数据且无空值，无需额外处理缺失值问题
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   订单ID    500000 non-null  int64  
 1   用户ID    500000 non-null  int64  
 2   下单时间    500000 non-null  object 
 3   商品大类    500000 non-null  object 
 4   商品子类    500000 non-null  object 
 5   商品名称    500000 non-null  object 
 6   单价      500000 non-null  float64
 7   购买数量    500000 non-null  int64  
 8   订单总额    500000 non-null  float64
 9   收货省份    500000 non-null  object 
 10  收货城市    500000 non-null  object 
 11  性别      500000 non-null  object 
 12  年龄      500000 non-null  int64  
 13  是否会员    500000 non-null  object 
dtypes: float64(2), int64(4), object(8)
memory usage: 53.4+ MB


In [4]:
data.columns  #列名

Index(['订单ID', '用户ID', '下单时间', '商品大类', '商品子类', '商品名称', '单价', '购买数量', '订单总额',
       '收货省份', '收货城市', '性别', '年龄', '是否会员'],
      dtype='object')

In [5]:
data[data.duplicated()].count()   #完全无重复数据

订单ID    0
用户ID    0
下单时间    0
商品大类    0
商品子类    0
商品名称    0
单价      0
购买数量    0
订单总额    0
收货省份    0
收货城市    0
性别      0
年龄      0
是否会员    0
dtype: int64

In [6]:
data.isnull().sum()    #再验证一遍，完全无空值

订单ID    0
用户ID    0
下单时间    0
商品大类    0
商品子类    0
商品名称    0
单价      0
购买数量    0
订单总额    0
收货省份    0
收货城市    0
性别      0
年龄      0
是否会员    0
dtype: int64

## 2.数据分析可视化
### 2.1 空间与时间维度的消费差异分析

#### 2.1.1 空间维度上的不同地区订单量和订单总额分析

In [7]:
#先导入需要的库
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker
from datetime import datetime

In [8]:
#转换表格中省份名称成pycharts能识别的省市名称
province_mapping = {
    # 直辖市（5个）
    '北京': '北京市',
    '上海': '上海市',
    '天津': '天津市',
    '重庆': '重庆市',
    # 省（23个）
    '河北': '河北省',
    '山西': '山西省',
    '辽宁': '辽宁省',
    '吉林': '吉林省',
    '黑龙江': '黑龙江省',
    '江苏': '江苏省',
    '浙江': '浙江省',
    '安徽': '安徽省',
    '福建': '福建省',
    '江西': '江西省',
    '山东': '山东省',
    '河南': '河南省',
    '湖北': '湖北省',
    '湖南': '湖南省',
    '广东': '广东省',
    '海南': '海南省',
    '四川': '四川省',
    '贵州': '贵州省',
    '云南': '云南省',
    '陕西': '陕西省',
    '甘肃': '甘肃省',
    '青海': '青海省',
    '台湾': '台湾省',
    # 自治区（5个）
    '内蒙古': '内蒙古自治区',
    '广西': '广西壮族自治区',
    '西藏': '西藏自治区',
    '宁夏': '宁夏回族自治区',
    '新疆': '新疆维吾尔自治区',
    # 特别行政区（2个）
    '香港': '香港特别行政区',
    '澳门': '澳门特别行政区'
}
data['收货省份'] = data['收货省份'].map(lambda x: province_mapping.get(x, x))

In [9]:
#各地区订单分布可视化需要用到‘订单总额’，‘订单编号’，‘收获省份’
order_area=data.groupby('收货省份').agg({'订单ID':'count','订单总额':lambda x: round(x.sum()/10000, 2)})
#生成销量和销售额字典
order_count= order_area.to_dict()['订单ID'] 
order_amount= order_area.to_dict()['订单总额']
#生成地图，添加视觉映射条
a=(
    Map()
    .add("订单量",[*order_count.items()],"china",is_map_symbol_show=False)
    .add("订单总额（万）",[*order_amount.items()],"china",is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="地区分布"),
        visualmap_opts=opts.VisualMapOpts(max_=30000),
    )
)
a.render('各地区订单量和订单总额.html')

'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\各地区订单量和订单总额.html'

直观区分区域差异，快速定位订单集中的核心区与潜力区，为区域营销资源分配、市场拓展提供地理维度数据支撑

#### 2.1.2时间维度商品销量和销售额走势

In [10]:
#先转换str成为datetime格式，方便下面进行dt操作
data['下单时间']=pd.to_datetime(data['下单时间'])

In [11]:
#取表中下单时间中的小时部分，统计每小时订单量
data['下单时间_小时'] = data['下单时间'].dt.hour
order_time=data.groupby(data['下单时间_小时']).agg({"订单ID":"count"}).to_dict()["订单ID"]
x = list(order_time.keys())
y = list(order_time.values())
#生成柱状图，添加峰值标注
b=(
    Bar()
    .add_xaxis(x)
    .add_yaxis("订单量",y)
    .set_global_opts(title_opts=opts.TitleOpts("每小时订单销量走势"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_='max',name='峰值')
            ]
        ),
    )
)
b.render("每小时订单销量走势.html")
#可见这个虚拟订单作者在生成订单时每小时均匀生成数据

'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\每小时订单销量走势.html'

In [12]:
#取表中下单时间中的年月日部分，统计订单量和订单销售量
data['下单日期_年月日'] = data['下单时间'].dt.strftime('%Y-%m-%d')
sale_time=data.groupby("下单日期_年月日").agg({"订单ID":"count","订单总额":"sum"})
order_count_=sale_time.to_dict()["订单ID"]
order_amount_=sale_time.to_dict()["订单总额"]
#生成两个折线图，添加峰值标注
c=(
    Line()
    .add_xaxis(list(order_count_.keys()))
    .add_yaxis(
        series_name="订单量",
        y_axis=list(order_count_.values()),
        yaxis_index=0, 
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="max", name="最大值")]
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每日订单量走势"),
    )
)
d=(
    Line()
    .add_xaxis(list(order_count_.keys()))
    .add_yaxis(
        series_name="订单总额",
        y_axis=list(order_amount_.values()),
        yaxis_index=1, 
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="max", name="最大值")]
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每日销售额走势"),
    )
)
c.render("每日订单量走势.html")
c.render("每日销售额走势.html")

'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\每日销售额走势.html'

柱状图显示虚拟数据每小时订单量均匀分布，无高峰，仅标注峰值。  
日期维度：“下单时间” 转 “年月日” 后，折线图呈现每日订单量与销售额，可识别最大值节点，但无显著周期性波动，反映数据时间维度生成逻辑平稳。

### 2.2销售业绩分析

#### 2.2.1整体销售概览

In [13]:
#总订单量、总销售额和平均客单价
result={}
result["总订单量"]=f'{data["订单ID"].count()/10000:.0f}万'
result["商品交易总额(GMV)"]=f'{data["订单总额"].sum()/10000/10000:.2f}亿'
result["平均客单价"]=f'{data["订单总额"].sum()/data["订单ID"].count():.2f}元'
result

{'总订单量': '50万', '商品交易总额(GMV)': '3.88亿', '平均客单价': '776.57元'}

总订单量 50 万单，总销售额 3.88 亿元，平均客单价 776.57 元。  
客单价契合阿迪达斯中高端定位，为对比不同维度销售表现、评估业绩达成提供基础参照

#### 2.2.2商品大类子类订单分析

In [14]:
#找出各商品大类的销量和销售额
result2=data.groupby("商品大类").agg({"订单ID":"count","订单总额":"sum"}).reset_index()
display(result2)
#生成商品大类表格
table=Table()
headers = ["商品大类", "订单销量", "订单总额（万）"]
rows=[]
for idx, row in result2.iterrows():
    amount = f"{row['订单总额']/10000:.2f}万"
    rows.append([row["商品大类"], row["订单ID"], amount])
table.add(headers,rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='商品大类订单分析')
)
table.render("商品大类订单分析.html")

,商品大类,订单ID,订单总额
0,运动服装,199697,1.598745e+08
1,运动装备,62768,1.521145e+07
2,运动配饰,71688,2.361641e+07
3,运动鞋,165847,1.895848e+08


'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\商品大类订单分析.html'

In [15]:
#找出各商品子类的销量和销售额
result3=data.groupby("商品子类").agg({"订单ID":"count","订单总额":"sum"}).reset_index()
display(result3)
#生成商品子类表格
table=Table()
headers = ["商品子类", "订单销量", "订单总额（万）"]
rows=[]
for idx, row in result3.iterrows():
    amount = f"{row['订单总额']/10000:.2f}万"
    rows.append([row["商品子类"], row["订单ID"], amount])
table.add(headers,rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='商品子类订单分析')
)
table.render("商品子类订单分析.html")

,商品子类,订单ID,订单总额
0,上衣,64019,36293670.85
1,下装,57912,26328772.40
2,休闲鞋,38364,36729779.56
3,其他配饰,14265,3444454.51
4,包袋,14379,8048320.99
5,外套,36018,39497886.89
6,套装,41748,57754132.83
7,帽子,14609,3834942.42
8,户外鞋,21837,30744180.89
9,手套,14227,5039691.04


'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\商品子类订单分析.html'

In [16]:
#找出销量最高和销售额最高的五个商品
result3=data.groupby("商品名称").agg(销量=("订单ID", "count"),销售额=("订单总额", "sum")).reset_index()
top5_sales = result3.sort_values(by="销量", ascending=False).head(5)
top5_revenue = result3.sort_values(by="销售额", ascending=False).head(5)
print(top5_sales[["商品名称", "销量"]].values)
#销量Top5表格
table_sales = Table()
headers_sales = ["商品名称", "销量"]
rows_sales = [list(row) for row in top5_sales[["商品名称", "销量"]].values]
table_sales.add(headers_sales, rows_sales)
table_sales.set_global_opts(title_opts=ComponentTitleOpts(title='销量TOP5'))
table_sales.render("销量Top5.html")

# 销售额Top5表格（单位：万）
table_revenue = Table()
headers_revenue = ["商品名称", "销售额（单位：万）"]
rows_revenue = [
    [row[0], f"{row[1]/10000:.2f}"] 
    for row in top5_revenue[["商品名称", "销售额"]].values
]
table_revenue.add(headers_revenue, rows_revenue)
table_revenue.set_global_opts(title_opts=ComponentTitleOpts(title='销售额TOP5'))
table_revenue.render("销售额Top5.html")

[['训练套装' 8429]
 ['运动套装' 8411]
 ['足球套装' 8406]
 ['运动长裤' 8393]
 ['阿尔法训练师' 8379]]


'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\销售额Top5.html'

大类：运动服装（199697 单）、运动鞋（165847 单）订单量居前，贡献主要销售额；  
运动装备（62768 单）、配饰（71688 单）较少。  
子类：套装（41748 单）、上衣（64019 单）、休闲鞋（38364 单）突出，训练 / 运动套装等进销量 TOP5，  
且销售额 TOP5 与销量 TOP5 重合，为采购、库存、促销选品指路。

### 2.3用户分析

#### 2.3.1用户性别分布及消费偏好关联

In [17]:
#先看用户男女性别比例
gender=data.groupby("性别")["性别"].count().reset_index(name="人数")
total=gender["人数"].sum()
gender["占比"]=gender["人数"].apply(lambda x: f"{x/total*100:.2f}%")
display(gender)

#接下来看用户性别是否与消费行为有明显关联
data_gender=data.groupby("性别").agg(销量=("订单ID","count"),销售额=("订单总额","sum"))
counts=data_gender.sort_values(by="销量",ascending=False)
amounts=data_gender.sort_values(by="销售额",ascending=False)
display(counts)#销量降序排序
display(amounts)#销售额降序排序


,性别,人数,占比
0,女,253509,50.70%
1,男,246491,49.30%


,销量,销售额
性别,,
女,253509,1.869798e+08
男,246491,2.013072e+08


,销量,销售额
性别,,
男,246491,2.013072e+08
女,253509,1.869798e+08


消费上，女性订单量略高，男性销售额 2.01 亿元高于女性 1.87 亿元，差异小，性别对消费偏好影响不明显，无需针对性核心策略。

#### 2.3.2用户年龄分布及消费偏好关联

In [18]:
#先看总数据中用户各年龄占总人数的比值
age=data.groupby("年龄")["年龄"].count().reset_index(name="人数")
total=age["人数"].sum()
age["占比"]=age["人数"].apply(lambda x: f"{x/total*100:.2f}%")
age=age.sort_values(by="占比",ascending=False)
display(age)

##接下来看用户年龄是否与消费行为有明显关联
age_data=data.groupby("年龄").agg(销量=("订单ID","count"),销售额=("订单总额","sum"))
data1=age_data.sort_values(by="销量",ascending=False)
data2=age_data.sort_values(by="销售额",ascending=False)
display(data1)#销量降序排序
display(data2)#销售额降序排序

,年龄,人数,占比
0,18,25684,5.14%
5,23,25460,5.09%
2,20,24513,4.90%
3,21,24199,4.84%
1,19,24208,4.84%
13,31,24013,4.80%
14,32,23820,4.76%
15,33,23471,4.69%
12,30,23095,4.62%
6,24,22347,4.47%


,销量,销售额
年龄,,
18,25684,21730284.14
23,25460,21488694.75
20,24513,20808005.99
19,24208,20384547.74
21,24199,20573639.36
31,24013,17186195.92
32,23820,17206178.92
33,23471,16605128.98
30,23095,16491729.30


,销量,销售额
年龄,,
18,25684,21730284.14
23,25460,21488694.75
20,24513,20808005.99
21,24199,20573639.36
19,24208,20384547.74
24,22347,18979487.14
32,23820,17206178.92
31,24013,17186195.92
22,20630,17131904.88


销量、销售额前几的年龄组均为 18-34 岁，与人数分布一致，无特殊偏好，品牌应聚焦年轻群体营销。

#### 2.3.3 构建RFM模型对用户进行分层并可视化

#### 1.准备工作

In [19]:
#先再导一遍库，并且过滤掉非必要warning
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyecharts import options as opts
from pyecharts.charts import Bar, Pie, Scatter, Radar
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
import warnings
warnings.filterwarnings('ignore')

# 转换日期格式
data['下单时间'] = pd.to_datetime(data['下单时间'])
print(f"数据时间范围：{data['下单时间'].min()} 至 {data['下单时间'].max()}")

数据时间范围：2024-01-01 00:00:47.416583 至 2024-12-30 23:58:03.794850


#### 2.数据预处理

In [20]:
# 确定基准日期（数据最后日期+1天）
max_date = data['下单时间'].max()
reference_date = max_date + timedelta(days=1)
print(f"RFM计算基准日期：{reference_date}")

# 按用户聚合得到RFM
rfm_df = data.groupby('用户ID').agg({
    '下单时间': lambda x: (reference_date - x.max()).days,  # Recency：最近消费天数
    '订单ID': 'count',  # Frequency：消费次数
    '订单总额': 'sum'  # Monetary：消费总金额
}).reset_index()

rfm_df.columns = ['用户ID', 'Recency', 'Frequency', 'Monetary']

#处理极端值（界定为>=95%的数据）
freq_95 = rfm_df['Frequency'].quantile(0.95)
mon_95 = rfm_df['Monetary'].quantile(0.95)
rfm_df['Frequency'] = rfm_df['Frequency'].apply(lambda x: freq_95 if x > freq_95 else x)
rfm_df['Monetary'] = rfm_df['Monetary'].apply(lambda x: mon_95 if x > mon_95 else x)
display(rfm_df)

RFM计算基准日期：2024-12-31 23:58:03.794850


,用户ID,Recency,Frequency,Monetary
0,1,8,117.0,94335.0275
1,2,2,101.0,78732.4700
2,3,1,60.0,46850.4700
3,4,3,28.0,28950.4000
4,5,10,64.0,35163.7100
...,...,...,...,...
9995,9996,2,46.0,42835.2700
9996,9997,7,37.0,26766.1800
9997,9998,19,66.0,44340.9800
9998,9999,1,39.0,29447.8100


In [21]:
# RFM评分与用户分层  
rfm_score = rfm_df.copy()

# Recency评分（值越小分数越高）
rfm_score['R_Score'] = pd.qcut(
    rfm_score['Recency'], q=5, labels=[5,4,3,2,1], duplicates='drop'
).astype(int)

# Frequency评分（值越大分数越高）
rfm_score['F_Score'] = pd.qcut(
    rfm_score['Frequency'], q=5, labels=[1,2,3,4,5], duplicates='drop'
).astype(int)

# Monetary评分（值越大分数越高）
rfm_score['M_Score'] = pd.qcut(
    rfm_score['Monetary'], q=5, labels=[1,2,3,4,5], duplicates='drop'
).astype(int)

#RFM总评分（15分满分）
rfm_score['RFM_Total'] = rfm_score['R_Score'] + rfm_score['F_Score'] + rfm_score['M_Score']
display(rfm_score) #显示一下rfm_score表中的评分情况

#用户分层
def user_segment(row):
    """根据RFM评分判断用户类型"""
    R_flag = 'H' if row['R_Score'] >= 3 else 'L'
    F_flag = 'H' if row['F_Score'] >= 3 else 'L'
    M_flag = 'H' if row['M_Score'] >= 3 else 'L'
    
    if R_flag == 'H' and F_flag == 'H' and M_flag == 'H':
        return '高价值用户（核心）'
    elif R_flag == 'H' and F_flag == 'H' and M_flag == 'L':
        return '高频忠诚用户'
    elif R_flag == 'H' and F_flag == 'L' and M_flag == 'H':
        return '高客单潜力用户'
    elif R_flag == 'H' and F_flag == 'L' and M_flag == 'L':
        return '新用户'
    elif R_flag == 'L' and F_flag == 'H' and M_flag == 'H':
        return '回流用户'
    elif R_flag == 'L' and F_flag == 'H' and M_flag == 'L':
        return '一般忠诚用户'
    elif R_flag == 'L' and F_flag == 'L' and M_flag == 'H':
        return '高客单沉睡用户'
    else:
        return '流失用户'

rfm_score['用户类型'] = rfm_score.apply(user_segment, axis=1)
#得到新的一列["用户类型"]
display(rfm_score)

,用户ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Total
0,1,8,117.0,94335.0275,2,5,5,12
1,2,2,101.0,78732.4700,5,5,5,15
2,3,1,60.0,46850.4700,5,4,4,13
3,4,3,28.0,28950.4000,4,2,3,9
4,5,10,64.0,35163.7100,2,4,3,9
...,...,...,...,...,...,...,...,...
9995,9996,2,46.0,42835.2700,5,3,4,12
9996,9997,7,37.0,26766.1800,2,3,3,8
9997,9998,19,66.0,44340.9800,1,4,4,9
9998,9999,1,39.0,29447.8100,5,3,3,11


,用户ID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Total,用户类型
0,1,8,117.0,94335.0275,2,5,5,12,回流用户
1,2,2,101.0,78732.4700,5,5,5,15,高价值用户（核心）
2,3,1,60.0,46850.4700,5,4,4,13,高价值用户（核心）
3,4,3,28.0,28950.4000,4,2,3,9,高客单潜力用户
4,5,10,64.0,35163.7100,2,4,3,9,回流用户
...,...,...,...,...,...,...,...,...,...
9995,9996,2,46.0,42835.2700,5,3,4,12,高价值用户（核心）
9996,9997,7,37.0,26766.1800,2,3,3,8,回流用户
9997,9998,19,66.0,44340.9800,1,4,4,9,回流用户
9998,9999,1,39.0,29447.8100,5,3,3,11,高价值用户（核心）


In [22]:
# 分层统计：数量+占比
segment_stats = rfm_score['用户类型'].value_counts().reset_index()
segment_stats.columns = ['用户类型', '用户数量']
segment_stats['用户占比(%)'] = (segment_stats['用户数量'] / segment_stats['用户数量'].sum() * 100).round(2)
display(segment_stats)

# 分层统计：消费价值
segment_value = rfm_score.groupby('用户类型')['Monetary'].agg(['mean', 'sum']).reset_index()
segment_value.columns = ['用户类型', '平均消费金额', '总消费金额']
segment_value[['平均消费金额', '总消费金额']] = segment_value[['平均消费金额', '总消费金额']].round(2)
segment_value['总消费占比(%)'] = (segment_value['总消费金额'] / segment_value['总消费金额'].sum() * 100).round(2)
display(segment_value)

,用户类型,用户数量,用户占比(%)
0,高价值用户（核心）,4387,43.87
1,流失用户,2363,23.63
2,新用户,1311,13.11
3,回流用户,1268,12.68
4,高频忠诚用户,196,1.96
5,高客单沉睡用户,181,1.81
6,高客单潜力用户,164,1.64
7,一般忠诚用户,130,1.30


,用户类型,平均消费金额,总消费金额,总消费占比(%)
0,一般忠诚用户,24148.97,3.139366e+06,0.82
1,回流用户,46023.81,5.835820e+07,15.30
2,新用户,16292.49,2.135945e+07,5.60
3,流失用户,13478.16,3.184889e+07,8.35
4,高价值用户（核心）,57467.73,2.521109e+08,66.08
5,高客单沉睡用户,29264.65,5.296901e+06,1.39
6,高客单潜力用户,28976.58,4.752160e+06,1.25
7,高频忠诚用户,23764.58,4.657859e+06,1.22


##### 从上面dataframe表中可以看出“高价值用户（核心）”是最大用户群体以及绝对消费主力

#### 3.数据可视化

In [23]:
#数据可视化
#用户类型数量饼图
pie = (
    Pie()
    .add("用户数量", list(zip(segment_stats['用户类型'], segment_stats['用户数量'])), radius=["30%", "70%"])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="RFM用户分层数量分布"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_left="10%")
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
)
pie.render("RFM用户数量分布饼图.html")

#消费金额柱状图
segment_sorted = segment_value.sort_values('总消费金额')
bar = (
    Bar()
    .add_xaxis(segment_sorted['用户类型'].tolist())
    .add_yaxis("总消费金额（万元）", [round(v/10000, 2) for v in segment_sorted['总消费金额']])
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各用户类型消费金额分布"),
        xaxis_opts=opts.AxisOpts(name="万元"),
        yaxis_opts=opts.AxisOpts(name="用户类型")
    )
)
bar.render("RFM用户消费金额柱状图.html")

'C:\\Users\\Lenovo\\Desktop\\数据分析\\Adidas虚拟数据\\RFM用户消费金额柱状图.html'

高价值用户（核心）占 43.87%（4387 人），贡献 66.08% 总消费；  
流失用户占 23.63%（2363 人），消费占比仅 8.35%；新用户（13.11%）、回流用户（12.68%）等占比均低于 15%，消费贡献有限。  
饼图（数量）、柱状图（金额）凸显高价值用户重要性，为用户运营指明方向。

## 结语
本次基于阿迪达斯虚拟电商数据集的分析，经 “数据理解 - 清洗 - 可视化 - 深度挖掘” 流程，形成核心结论与应用建议如下：
### 一、核心结论
#### 数据与业务基础：
50 万条订单数据无缺失、无重复，完整性良好；总订单量 50 万、商品交易总额(GMV) 3.88 亿，平均客单价 776.57 元，契合中高端运动品牌定位。
#### 时空规律：
地区消费呈 “区域集中化” 特征；每小时订单量因虚拟数据特性分布均匀，日级走势可结合促销、节假日进一步分析。
#### 商品表现：
运动服装（19.97 万单）与运动鞋（16.58 万单）为核心品类，合计占超 70% 订单量；套装、上衣、跑步鞋是热销子类，“训练套装”“运动套装” 等套装类商品位列销量 TOP3。
#### 用户特征：
性别分布均衡（女 50.70%、男 49.30%），且与消费偏好无显著关联；18-34 岁年轻群体为消费主力；RFM 分层显示，高价值用户（核心）占 43.87%，贡献 66.08% 总消费额，流失用户（23.63%）与新用户（13.11%）需重点运营。
### 二、业务建议
#### 商品运营：
聚焦核心品类，加大热销子类备货与新品开发；针对 TOP 商品设计组合优惠，提升连带销售率。
#### 用户运营：
为高价值用户提供专属权益，巩固忠诚度；用定向优惠券等激活流失用户；针对 18-34 岁群体，借社交媒体强化品牌认知。
#### 数据延伸：
补充促销、物流、用户评价等真实数据，深化关联分析；优化 RFM 模型参数，提升用户分层精准度。  
本次分析框架（时空 - 商品 - 用户三维分析、RFM 分层）可迁移至真实电商场景，为运动品牌精细化运营与数据决策提供参考。

bye~~~